<a href="https://colab.research.google.com/github/Srilekha-2000/Language-Translator/blob/main/Twitter_Sentiment_Analysis_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
train_data=pd.read_csv(r"train.csv", encoding='ISO-8859-1')

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

def most_used_words(train_data):
    tokens = word_tokenize(train_data)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(tokens)))
    return sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

mw = most_used_words(train_data.SentimentText.str.cat())
most_words = []
for w in mw:
    if len(most_words) == 1000:
        break
    if w in stopwords.words("english"):
        continue
    else:
        most_words.append(w)

There is 129018 different words


In [ ]:

from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

def stem_tokenize(train_data):
    stemmer = SnowballStemmer("english")
    stemmer = WordNetLemmatizer()
    return [stemmer.lemmatize(token) for token in word_tokenize(train_data)]

def lemmatize_tokenize(train_data):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(train_data)]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train_data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import nltk
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split

sentiments = train_data['Sentiment']
tweets = train_data['SentimentText']

vectorizer = TfidfVectorizer(tokenizer=lemmatize_tokenize, ngram_range=(2,2))
learn_data, test_data, sentiments_learning, sentiments_test = train_test_split(tweets, sentiments, test_size=0.3)
learning_data = vectorizer.fit_transform(learn_data)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

lr = LogisticRegression()
bnb = BernoulliNB()
mnb = MultinomialNB()

models = {
    'logitic regression': lr,
    'bernoulliNB': bnb,
    'multinomialNB': mnb,
}
for model in models.keys():
    scores = cross_val_score(models[model], learning_data, sentiments_learning, scoring="f1", cv=10)
    print("===", model, "===")
    print("scores = ", scores)
    models[model].fit(learning_data, sentiments_learning)
    print("score on the learning data (accuracy) = ", accuracy_score(models[model].predict(learning_data), sentiments_learning))
    print("")

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

=== logitic regression ===
scores =  [0.78480726 0.78802368 0.79096173 0.7821368  0.78231293 0.78182646
 0.78343732 0.78541596 0.7893898  0.784563  ]


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


score on the learning data (accuracy) =  0.888558692421991

=== bernoulliNB ===
scores =  [0.76828465 0.77119577 0.77387486 0.76488062 0.76511986 0.77162744
 0.7693662  0.76701167 0.7696697  0.76727754]
score on the learning data (accuracy) =  0.9115470339467368

=== multinomialNB ===
scores =  [0.79764732 0.79805408 0.8001355  0.79142565 0.79117881 0.7926593
 0.79486313 0.78778316 0.79621067 0.793694  ]
score on the learning data (accuracy) =  0.9552806034975426



In [ ]:
mnb.fit(learning_data, sentiments_learning)

MultinomialNB()

In [ ]:
testing_data = vectorizer.transform(test_data)
mnb.score(testing_data, sentiments_test)

0.7405073840717405

In [ ]:
model = MultinomialNB()
model.fit(learning_data, sentiments_learning)
tweet = pd.Series([input(),])
tweet = vectorizer.transform(tweet)
proba = model.predict_proba(tweet)[0]
print("The probability that this tweet is sad is:", proba[0])
print("The probability that this tweet is happy is:", proba[1])

0
The probability that this tweet is sad is: 0.43379243342096246
The probability that this tweet is happy is: 0.5662075665790376
